In [1]:
import trimesh
import potpourri3d as pp3d
import numpy as np
import pyvista as pv
from scipy.spatial import KDTree

In [2]:
def unify_vertices(source_file_path, tolerance=1e-5):
    mesh = trimesh.load(source_file_path)
    
    vertices = mesh.vertices
    faces = mesh.faces

    # Step 1: Find unique vertices within the given tolerance
    kdtree = KDTree(vertices)
    unique_indices = {}
    canonical_vertices = []
    
    for i, vertex in enumerate(vertices):
        if i in unique_indices:
            continue  # Skip if already mapped to a canonical vertex
        
        # Find all vertices within the tolerance distance
        duplicate_indices = kdtree.query_ball_point(vertex, tolerance)
        
        # Use the first duplicate as the "canonical" vertex and map all duplicates to it
        canonical_index = len(canonical_vertices)
        canonical_vertices.append(vertex)
        
        for idx in duplicate_indices:
            unique_indices[idx] = canonical_index

    # Step 2: Update faces to use only unique vertices
    new_faces = np.array([[unique_indices[vi] for vi in face] for face in faces])

    # Step 3: Create the new unified mesh
    unified_mesh = trimesh.Trimesh(vertices=np.array(canonical_vertices), faces=new_faces)

    output_path = source_file_path[:-4] + '_fixed.obj'
    unified_mesh.export(output_path)
    print(f'Mesh unified and saved to {output_path}')

In [3]:
def remove_duplicate_edges(source_file_path):
    #######add-in########
    mesh = trimesh.load(source_file_path)
    ################################
    edges = mesh.edges
    faces = mesh.faces
    sorted_edges = np.sort(edges, axis=1)

    # Find unique edges
    _, unique_indices = np.unique(sorted_edges, axis=0, return_index=True)
    unique_edges = edges[unique_indices]

    # Find the vertices used by unique edges
    unique_vertices = np.unique(unique_edges)

    # Remap vertices for the unique edges
    vertex_map = {old_index: new_index for new_index, old_index in enumerate(unique_vertices)}
    remapped_vertices = np.array([vertex_map[vertex] for vertex in unique_vertices])


    # Recreate the mesh using the unique edges and remapped vertices
    new_mesh = trimesh.Trimesh(vertices=mesh.vertices[remapped_vertices], faces=faces, process=False)

    output_path = source_file_path[:-4] + '_edges_cleaned.obj'
    new_mesh.export(output_path)
    print(f'Mesh cleaned and saved to {output_path}')


In [4]:
def remove_faces_by_indices(mesh, face_indices):
    # Convert face_indices to a set for fast exclusion
    face_indices_set = set(face_indices)

    # Create a mask that selects all faces except those we want to remove
    mask = [i not in face_indices_set for i in range(len(mesh.faces))]

    # Filter the faces array
    new_faces = mesh.faces[mask]

    # Recreate the mesh with the remaining faces
    new_mesh = trimesh.Trimesh(vertices=mesh.vertices, faces=new_faces, process=False)
    
    return new_mesh

In [5]:
# https://github.com/nmwsharp/robust-laplacians-py/issues/1
def add_random_jitter(source_file_path, level=1e-5):
    mesh = trimesh.load(source_file_path)
    jittered_vertices = mesh.vertices + np.random.normal(0, level, size= mesh.vertices.shape)
    unified_mesh = trimesh.Trimesh(vertices=jittered_vertices, faces= mesh.faces)
    output_path = source_file_path[:-4] + '_jittered.obj'
    unified_mesh.export(output_path)
    print(f'Mesh jittered and saved to {output_path}')

In [33]:
import trimesh
import potpourri3d as pp3d
import numpy as np
import pyvista as pv

# Load the mesh
mesh = trimesh.load('spot_fixed_edges_cleaned.obj')

# Set up the PyVista plotter
plotter = pv.Plotter()

# Multisource: source 1 + source 2
multisource_vertex= [0, 100]
heat_solver_1 = pp3d.MeshHeatMethodDistanceSolver(mesh.vertices, mesh.faces)
distances_multisource = heat_solver_1.compute_distance_multisource(multisource_vertex)

# First Source
source_vertex_1 = multisource_vertex[0]
heat_solver_1 = pp3d.MeshHeatMethodDistanceSolver(mesh.vertices, mesh.faces)
distances_1 = heat_solver_1.compute_distance(source_vertex_1)

# Second Source
source_vertex_2 = multisource_vertex[1]
heat_solver_2 = pp3d.MeshHeatMethodDistanceSolver(mesh.vertices, mesh.faces)
distances_2 = heat_solver_2.compute_distance(source_vertex_2)

# # Normalize distances for visualization
# normalized_distances_1 = (distances_1 - distances_1.min()) / (distances_1.max() - distances_1.min())
# normalized_distances_2 = (distances_2 - distances_2.min()) / (distances_2.max() - distances_2.min())

# Combine Distance Information for Visualization
normalized_distances_multisource = (distances_multisource - distances_multisource.min()) / (distances_multisource.max() - distances_multisource.min())

# # Prepare the mesh for PyVista
# faces_with_sizes = np.hstack([np.full((mesh.faces.shape[0], 1), 3), mesh.faces]).flatten()
# pv_mesh = pv.PolyData(mesh.vertices, faces_with_sizes)

# Add the combined geodesic distance visualization
plotter.add_mesh(pv_mesh, scalars=normalized_distances_multisource, cmap='viridis', show_edges=True)

# Prepare the mesh for PyVista
faces_with_sizes = np.hstack([np.full((mesh.faces.shape[0], 1), 3), mesh.faces]).flatten()
pv_mesh = pv.PolyData(mesh.vertices, faces_with_sizes)

# Find Cut Locus Points (where distances are approximately equal)
tolerance = 0.05  # Adjust for precision
cut_locus_vertices = np.where(np.abs(distances_1 - distances_2) < tolerance)[0]

plotter.add_points(mesh.vertices[source_vertex_1].reshape(1, 3), color="red", render_points_as_spheres=True, point_size=10, label="Source 1")
plotter.add_points(mesh.vertices[source_vertex_2].reshape(1, 3), color="red", render_points_as_spheres=True, point_size=10, label="Source 2")

# Highlight the cut locus points
plotter.add_points(mesh.vertices[cut_locus_vertices], color="blue", render_points_as_spheres=True, point_size=8, label="Cut Locus")

# Vector Heat Method for multisource
vector_solver_multisource = pp3d.MeshVectorHeatSolver(mesh.vertices, mesh.faces)
multisource_vector = [(1, 0), (0, 1)]

# Transport tangent vector for multisource
intrinsic_multisource = vector_solver_multisource.transport_tangent_vectors(multisource_vertex, multisource_vector)
basisX_multisource, basisY_multisource, _ = vector_solver_multisource.get_tangent_frames()
ext3D_multisource = intrinsic_multisource[:, 0, np.newaxis] * basisX_multisource + intrinsic_multisource[:, 1, np.newaxis] * basisY_multisource

# Add vector transport visualization for multisource
plotter.add_arrows(mesh.vertices, ext3D_multisource, mag=0.1, color="yellow", label="Vectors from Source 1 and Source 2")

# Add legend and show the plot
plotter.add_legend()
plotter.show(title="Cut Locus Visualization with Vector Heat Method")


Widget(value='<iframe src="http://localhost:64487/index.html?ui=P_0x231ffd349a0_26&reconnect=auto" class="pyvi…

In [35]:
import trimesh
import potpourri3d as pp3d
import numpy as np
import pyvista as pv

# Load the mesh
mesh = trimesh.load('spot_fixed_edges_cleaned.obj')

# Set up the PyVista plotter
plotter = pv.Plotter()

# Multisource: source 1 + source 2
multisource_vertex= [0, 100]
heat_solver_1 = pp3d.MeshHeatMethodDistanceSolver(mesh.vertices, mesh.faces)
distances_multisource = heat_solver_1.compute_distance_multisource(multisource_vertex)

# First Source
source_vertex_1 = multisource_vertex[0]
heat_solver_1 = pp3d.MeshHeatMethodDistanceSolver(mesh.vertices, mesh.faces)
distances_1 = heat_solver_1.compute_distance(source_vertex_1)

# Second Source
source_vertex_2 = multisource_vertex[1]
heat_solver_2 = pp3d.MeshHeatMethodDistanceSolver(mesh.vertices, mesh.faces)
distances_2 = heat_solver_2.compute_distance(source_vertex_2)

# # Normalize distances for visualization
# normalized_distances_1 = (distances_1 - distances_1.min()) / (distances_1.max() - distances_1.min())
# normalized_distances_2 = (distances_2 - distances_2.min()) / (distances_2.max() - distances_2.min())

# Combine Distance Information for Visualization
normalized_distances_multisource = (distances_multisource - distances_multisource.min()) / (distances_multisource.max() - distances_multisource.min())

# # Prepare the mesh for PyVista
# faces_with_sizes = np.hstack([np.full((mesh.faces.shape[0], 1), 3), mesh.faces]).flatten()
# pv_mesh = pv.PolyData(mesh.vertices, faces_with_sizes)

# Add the combined geodesic distance visualization
plotter.add_mesh(pv_mesh, scalars=normalized_distances_multisource, cmap='viridis', show_edges=True)

# Prepare the mesh for PyVista
faces_with_sizes = np.hstack([np.full((mesh.faces.shape[0], 1), 3), mesh.faces]).flatten()
pv_mesh = pv.PolyData(mesh.vertices, faces_with_sizes)

# Find Cut Locus Points (where distances are approximately equal)
tolerance = 0.05  # Adjust for precision
cut_locus_vertices = np.where(np.abs(distances_1 - distances_2) < tolerance)[0]

plotter.add_points(mesh.vertices[source_vertex_1].reshape(1, 3), color="red", render_points_as_spheres=True, point_size=10, label="Source 1")
plotter.add_points(mesh.vertices[source_vertex_2].reshape(1, 3), color="red", render_points_as_spheres=True, point_size=10, label="Source 2")

# Highlight the cut locus points
plotter.add_points(mesh.vertices[cut_locus_vertices], color="blue", render_points_as_spheres=True, point_size=8, label="Cut Locus")

# Vector Heat Method for multisource
vector_solver_multisource = pp3d.MeshVectorHeatSolver(mesh.vertices, mesh.faces)
multisource_vector = [(1, 0), (0, 1)]

# # Transport tangent vector for multisource
# intrinsic_multisource = vector_solver_multisource.transport_tangent_vectors(multisource_vertex, multisource_vector)
# basisX_multisource, basisY_multisource, _ = vector_solver_multisource.get_tangent_frames()
# ext3D_multisource = intrinsic_multisource[:, 0, np.newaxis] * basisX_multisource + intrinsic_multisource[:, 1, np.newaxis] * basisY_multisource

# # Add vector transport visualization for multisource
# plotter.add_arrows(mesh.vertices, ext3D_multisource, mag=0.1, color="yellow", label="Vectors from Source 1 and Source 2")

# Add legend and show the plot
plotter.add_legend()
plotter.show(title="Cut Locus Visualization with Vector Heat Method")


Widget(value='<iframe src="http://localhost:64487/index.html?ui=P_0x231ff5a76a0_28&reconnect=auto" class="pyvi…